# 다중 레이블 분류 (Multi-label Classification)

이 노트북은 하나의 샘플이 여러 개의 레이블(클래스)을 동시에 가질 수 있는 **다중 레이블 분류** 문제를 다룹니다.

예를 들어, 영화 장르를 예측하는 문제에서 하나의 영화는 '코미디', '액션', '로맨스' 등 여러 장르에 속할 수 있습니다.

**학습 과정:**
1.  `scikit-learn`을 사용하여 다중 레이블 분류를 위한 가상 데이터셋 생성
2.  데이터를 훈련 세트와 테스트 세트로 분할
3.  `MultiOutputClassifier`와 `SVC`(서포트 벡터 머신)를 사용하여 모델 구성 및 훈련
4.  모델 성능 평가 (Hamming Loss, Jaccard Score, F1-Score 등)

### 1. 라이브러리 임포트

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss, jaccard_score, f1_score

### 2. 다중 레이블 데이터 생성
`make_multilabel_classification` 함수를 사용하여 가상의 데이터셋을 생성합니다.

- `n_samples`: 생성할 샘플의 수
- `n_features`: 각 샘플의 특성(feature) 수
- `n_classes`: 전체 가능한 레이블의 종류
- `n_labels`: 각 샘플이 평균적으로 가질 레이블의 수

In [ ]:
X, y = make_multilabel_classification(n_samples=100, n_features=20, n_classes=5,
                                      n_labels=2, random_state=42)

print("특성(X) 데이터 형태:", X.shape)
print("레이블(y) 데이터 형태:", y.shape)
print("
첫 5개 샘플의 레이블 (이진 행렬):")
print(y[:5])

### 3. 훈련 및 테스트 세트 분리
생성된 데이터를 모델 학습을 위한 훈련 세트와 성능 평가를 위한 테스트 세트로 분리합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"훈련 데이터: {X_train.shape}, {y_train.shape}")
print(f"테스트 데이터: {X_test.shape}, {y_test.shape}")

### 4. 모델 구성 및 훈련

다중 레이블 분류를 위해 `MultiOutputClassifier`를 사용합니다. 이 분류기는 각 레이블(클래스)에 대해 독립적인 분류기를 하나씩 훈련시키는 래퍼(wrapper)입니다.

- **기본 분류기(Base Estimator)**: 여기서는 `SVC`(서포트 벡터 분류기)를 사용합니다.
- **`MultiOutputClassifier`**: `SVC`를 각 레이블에 대해 개별적으로 학습하고 예측을 수행합니다.

In [ ]:
# 각 레이블을 예측할 기본 모델로 SVC를 설정합니다.
base_svm = SVC(kernel='linear', probability=True, random_state=42)

# MultiOutputClassifier로 SVC를 감싸 다중 레이블 문제에 적용합니다.
# n_jobs=-1은 가능한 모든 CPU 코어를 사용하여 병렬로 훈련을 진행하라는 의미입니다.
multi_label_svm = MultiOutputClassifier(base_svm, n_jobs=-1)

print("모델 훈련 시작...")
multi_label_svm.fit(X_train, y_train)
print("모델 훈련 완료.")

### 5. 예측
훈련된 모델을 사용하여 테스트 데이터의 레이블을 예측합니다.

In [ ]:
y_pred = multi_label_svm.predict(X_test)

print("실제 레이블 (첫 5개):")
print(y_test[:5])
print("
예측된 레이블 (첫 5개):")
print(y_pred[:5])

### 6. 모델 평가
다중 레이블 분류는 단일 레이블 분류와 다른 평가 지표를 사용합니다.

#### 6.1. 햄밍 손실 (Hamming Loss)
전체 레이블 중에서 잘못 예측된 레이블의 비율을 나타냅니다. **0에 가까울수록** 모델 성능이 좋다는 것을 의미합니다.

(예: 샘플 1의 실제 레이블이 `[1, 0, 1]`이고 예측이 `[1, 1, 1]`이라면, 3개 레이블 중 1개가 잘못 예측되었으므로 이 샘플의 Hamming Loss는 1/3 입니다.)

In [ ]:
h_loss = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {h_loss:.4f}")

#### 6.2. 자카드 점수 (Jaccard Score / Exact Match Ratio)
한 샘플에 대한 모든 레이블을 **완벽하게** 예측한 경우에만 정답으로 인정하는 엄격한 지표입니다. `average='samples'` 옵션은 각 샘플별 자카드 점수의 평균을 계산합니다.

**1에 가까울수록** 성능이 좋습니다.

In [ ]:
# average='samples'는 각 샘플에 대해 정확히 일치하는 레이블의 비율을 계산한 후 평균을 냅니다.
jaccard_similarity = jaccard_score(y_test, y_pred, average='samples')
print(f"Jaccard Score (samples average): {jaccard_similarity:.4f}")

#### 6.3. F1-점수 (F1-Score)
정밀도(Precision)와 재현율(Recall)의 조화 평균으로, 다중 레이블 분류에서 가장 널리 사용되는 지표 중 하나입니다.

- `average='micro'`: 모든 레이블의 예측 결과를 (TP, FP, FN) 하나로 합쳐서 계산합니다. 레이블 간의 불균형이 심할 때 유용합니다.
- `average='macro'`: 각 레이블별 F1-점수를 각각 계산한 후, 그 점수들의 산술 평균을 냅니다. 모든 레이블을 동등하게 중요하다고 간주합니다.

**1에 가까울수록** 성능이 좋습니다.

In [ ]:
# micro: 전체 TP, FP, FN을 합산하여 계산 (레이블 불균형에 덜 민감)
f1_micro = f1_score(y_test, y_pred, average='micro')
print(f"Micro F1-score: {f1_micro:.4f}")

# macro: 각 레이블에 대한 F1-score를 계산한 후 평균 (레이블 불균형에 민감)
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-score: {f1_macro:.4f}")